In [1]:
from bs4 import BeautifulSoup


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time


In [3]:
driver = webdriver.Firefox()

In [4]:
driver.get("https://www.letemps.ch/monde?after=2024-09-25T07%3A56%3A01%2B02%3A00")

# if page can be scrolled:
#driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")


In [5]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [6]:
articles = soup.find_all('div', class_='post__text')

In [12]:
results = []
for i, article in enumerate(articles):
    try:
        # Check if title starts with "Réservé aux abonnés"
        title = article.find('h2', class_='post__title').text.strip() if article.find('h2', class_='post__title') else "No title"
        reserved = title.startswith("Réservé aux abonnés\n\n\n")
        
        # Replace the string if present
        title = title.replace("Réservé aux abonnés\n\n\n", "")
        
        link = f"https://www.letemps.ch/{article.find('a')['href']}" if article.find('a') else "No link"
        abstract = article.find('div', class_='post__lead').text.strip() if article.find('div', class_='post__lead') else "No abstract"
        author = article.find('div', class_="post__author").text.strip() if article.find('div', class_="post__author") else "No author"
        date = article.find('time', class_="post__publication-date").text if article.find('time', class_="post__publication-date") else "No date"
        
        # Append the result including the new boolean variable
        results.append({
            "title": title,
            "link": link,
            "abstract": abstract,
            "author": author,
            "date": date,
            "reserved": reserved  # Boolean for "Réservé aux abonnés"
        })
    except AttributeError as e:
        print(f"Error in article {i}: {e}")
        print(f"Article content: {article}")

# Output the results
results


[{'title': 'Malgré les soupçons, la Croix-Rouge russe est la bienvenue à Genève',
  'link': 'https://www.letemps.ch//monde/malgre-les-soupcons-la-croix-rouge-russe-est-la-bienvenue-a-geneve',
  'abstract': 'Un rapport interne de la Fédération internationale des sociétés de la Croix-Rouge et du Croissant-Rouge épargne la branche russe. Des révélations de la presse et l’Ukraine l’accusent pourtant de participer au transfert illégal d’enfants ukrainiens',
  'author': 'Frédéric Koller',
  'date': 'Publié le 27 septembre 2024 à 15:55. / Modifié le 27 septembre 2024 à 16:19.',
  'reserved': True},
 {'title': 'Au Japon, Shigeru Ishiba prend la tête du Parti libéral-démocrate au pouvoir et sera le prochain premier ministre',
  'link': 'https://www.letemps.ch//monde/asie-oceanie/shigeru-ishiba-prend-la-tete-de-son-parti-et-sera-le-prochain-premier-ministre-du-japon',
  'abstract': 'Sa cinquième tentative a été la bonne. Longtemps considéré comme l’éternel numéro deux de son parti, l’homme de 67